In [5]:
# Импортируем необходимые библиотеки
from pymongo import MongoClient
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
# from bertopic import BERTopic
import numpy as np
import re
from umap import UMAP
import hdbscan
from sklearn.cluster import HDBSCAN
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from nltk.corpus import stopwords
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score


In [2]:
from pymongo import MongoClient

# Подключение к MongoDB
client = MongoClient('localhost', 27017)
collection = client['VKR1']['ner_results']

# Загрузка документов
documents = list(collection.find())

# Формирование корпуса (только сущности)
corpus = []
for doc in documents:
    # Проверяем наличие поля entities
    if 'entities' in doc:
        # Собираем все текстовые сущности
        entities_texts = [entity['text'] for entity in doc['entities']]
        # Объединяем сущности в строку
        document_text = " ".join(entities_texts)
        corpus.append(document_text)

# Формирование корпуса с тегами типов сущностей
corpus_with_labels = []
for doc in documents:
    if 'entities' in doc:
        # Собираем сущности с их типами
        entities_with_labels = [f"{entity['label']}:{entity['text']}" for entity in doc['entities']]
        document_text = " ".join(entities_with_labels)
        corpus_with_labels.append(document_text)

# Проверяем результат
print("Корпус (только сущности):")
for i, doc in enumerate(corpus[:5], start=1):  # Покажем первые 5 документов
    print(f"Document {i}: {doc}")

print("\nКорпус с тегами типов сущностей:")
for i, doc in enumerate(corpus_with_labels[:5], start=1):  # Покажем первые 5 документов
    print(f"Document {i}: {doc}")


Корпус (только сущности):
Document 1: assurance slab big data platforms assurance techniques analytics big data assurance service level agreements slas model
Document 2: data integrity analytics services cloud technologies data analytics many processes extra information public cloud infrastructures private spark resource correct data faults
Document 3: smart asset management electric utilities condition monitoring assets amounts chunk rich data information is analytics machine learning techniques internet of things pathways engineering
Document 4: academic learning analytics higher education business value model such analytics design business value creation legal competition risk security predictive analytics higher education fields higher education institutions big data academic
Document 5: big data analytics smart data technologies compute cloud edge devices inmotion processing atre st processing event driven programming reduce sequential code

Корпус с тегами типов сущностей:
Docume

In [77]:
import json
from pymongo import MongoClient
from gensim.models.ldamodel import LdaModel
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Подключение к MongoDB
client = MongoClient('localhost', 27017)
db = client["VKR1"]
research_paper_collection = db["research_paper"]
ner_results_collection = db["ner_results"]

# Загрузка данных из MongoDB
def load_data():
    print("Загрузка данных из MongoDB...")
    try:
        research_papers = list(research_paper_collection.find({}, {"_id": 1, "yearPublished": 1}))
        ner_results = list(ner_results_collection.find({}, {"_id": 1, "entities": 1}))
        if not research_papers or not ner_results:
            raise ValueError("Коллекции MongoDB пусты или данные отсутствуют.")
        return pd.DataFrame(research_papers), pd.DataFrame(ner_results)
    except Exception as e:
        print(f"Ошибка при загрузке данных: {e}")
        return pd.DataFrame(), pd.DataFrame()

research_papers, ner_results = load_data()

if research_papers.empty or ner_results.empty:
    print("Данные не загружены. Проверьте подключение к MongoDB.")
    exit()

# Объединение данных
merged_data = pd.merge(research_papers, ner_results, on="_id")
print(f"Всего объединённых документов: {len(merged_data)}")

merged_data = merged_data[merged_data["yearPublished"].notnull()]
print(f"Документы с указанным годом публикации: {len(merged_data)}")

# Группировка текстов по категориям и годам
categories = ['ALGORITHM', 'TECHNOLOGY', 'METHOD', 'DATA', 'MODEL', 'APPLICATION', 'TASK', 'TOOL', 'FRAMEWORK', 'PARAMETER', 'METRIC']
tagged_corpus = {tag: {} for tag in categories}

print("Группировка текстов по категориям и годам...")
for _, row in tqdm(merged_data.iterrows(), total=len(merged_data)):
    year = row["yearPublished"]
    entities = row.get("entities", [])
    if entities:
        for entity in entities:
            tag = entity.get("label")
            text = entity.get("text")
            if tag in tagged_corpus and text:
                tagged_corpus[tag].setdefault(year, []).append(text)

# Нормализация когерентности с учётом направлений
def normalize_coherence(umass, cv, npmi, weights=(0.3, 0.4, 0.3)):
    """
    Нормализация метрик когерентности.
    u_mass: стремится к 0, преобразуется в отрицательное значение.
    cv, npmi: стремятся к максимуму, используются как есть.
    """
    # Инверсия u_mass
    umass_transformed = -umass
    metrics = [[umass_transformed], [cv], [npmi]]

    # Нормализация
    scaler = MinMaxScaler()
    normalized_metrics = scaler.fit_transform(metrics).flatten()

    # Взвешенное усреднение
    weighted_sum = sum(normalized_metrics[i] * weights[i] for i in range(len(weights)))
    return weighted_sum

# Расчёт когерентности
def calculate_coherence_gensim(texts, num_topics, alpha, beta):
    try:
        tokenized_texts = [text.split() for text in texts]
        dictionary = Dictionary(tokenized_texts)
        corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

        lda_model = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary, random_state=42, alpha=alpha, eta=beta)

        # Вычисление метрик когерентности
        coherence_umass = CoherenceModel(model=lda_model, texts=tokenized_texts, dictionary=dictionary, coherence='u_mass').get_coherence()
        coherence_cv = CoherenceModel(model=lda_model, texts=tokenized_texts, dictionary=dictionary, coherence='c_v').get_coherence()
        coherence_npmi = CoherenceModel(model=lda_model, texts=tokenized_texts, dictionary=dictionary, coherence='c_npmi').get_coherence()

        # Применение нормализации
        normalized_coherence = normalize_coherence(coherence_umass, coherence_cv, coherence_npmi)
        return normalized_coherence, lda_model
    except Exception as e:
        print(f"Ошибка при расчёте когерентности: {e}")
        return None, None

# Определение оптимального числа тем
param_grid = {
    'num_topics': range(4, 10),
    'alpha': ['symmetric', 0.1],
    'beta': ['symmetric', 0.1]
}

def find_optimal_topics_with_gensim(texts):
    if len(texts) < 5:
        return None, None, None

    best_score = -float('inf')
    best_params = None
    best_model = None

    for params in tqdm(ParameterGrid(param_grid)):
        coherence, model = calculate_coherence_gensim(
            texts, params['num_topics'], params['alpha'], params['beta']
        )
        if coherence and coherence > best_score:
            best_score = coherence
            best_params = params
            best_model = model

    return best_model, best_params, best_score

# Расчёт важности тем
def analyze_topic_trends(model, num_topics):
    topic_importance = {}
    for topic_id in range(num_topics):
        words = model.show_topic(topic_id, topn=10)  # Топ-10 слов для темы
        topic_importance[topic_id] = {word: weight for word, weight in words}
    return topic_importance

# Тематическое моделирование
def perform_topic_modeling_with_gensim():
    topic_keywords = {tag: {} for tag in categories}
    optimal_params = {tag: {} for tag in categories}
    topic_coherences = {tag: {} for tag in categories}
    topic_trends = {tag: {} for tag in categories}

    print("Проведение тематического моделирования...")
    for tag, yearly_texts in tqdm(tagged_corpus.items()):
        for year, texts in yearly_texts.items():
            if texts:
                model, params, score = find_optimal_topics_with_gensim(texts)
                if model is not None:
                    num_topics = params['num_topics']
                    topic_keywords[tag][year] = model.show_topics(num_topics=num_topics, formatted=False)
                    optimal_params[tag][year] = params
                    topic_coherences[tag][year] = score
                    topic_trends[tag][year] = analyze_topic_trends(model, num_topics)

    return topic_keywords, optimal_params, topic_coherences, topic_trends

# Запуск моделирования
topic_keywords, optimal_params, topic_coherences, topic_trends = perform_topic_modeling_with_gensim()


Загрузка данных из MongoDB...
Всего объединённых документов: 86564
Документы с указанным годом публикации: 86564
Группировка текстов по категориям и годам...


100%|██████████| 86564/86564 [00:06<00:00, 13418.58it/s]


Проведение тематического моделирования...


100%|██████████| 24/24 [16:08<00:00, 40.35s/it]

100%|██████████| 24/24 [17:17<00:00, 43.23s/it]

100%|██████████| 24/24 [17:11<00:00, 42.96s/it]

100%|██████████| 24/24 [16:54<00:00, 42.28s/it]

100%|██████████| 24/24 [17:23<00:00, 43.50s/it]

100%|██████████| 24/24 [18:56<00:00, 47.37s/it]

100%|██████████| 24/24 [23:06<00:00, 57.77s/it]

100%|██████████| 24/24 [21:21<00:00, 53.40s/it]

100%|██████████| 24/24 [20:16<00:00, 50.68s/it]

100%|██████████| 24/24 [21:47<00:00, 54.48s/it]

100%|██████████| 24/24 [22:33<00:00, 56.38s/it]

100%|██████████| 24/24 [20:16<00:00, 50.69s/it]

100%|██████████| 24/24 [17:44<00:00, 44.34s/it]

100%|██████████| 24/24 [16:47<00:00, 41.98s/it]

100%|██████████| 24/24 [16:31<00:00, 41.29s/it]

100%|██████████| 24/24 [16:44<00:00, 41.87s/it]

100%|██████████| 24/24 [17:05<00:00, 42.71s/it]

100%|██████████| 24/24 [16:56<00:00, 42.34s/it]

100%|██████████| 24/24 [16:55<00:00, 42.33s/it]

100%|██████████| 24/24 [16:32<00:00, 41.36s/it]

100%|██████████| 24/

In [79]:
topic_keywords

{'ALGORITHM': {2018: [(0,
    [('optimization', 0.04072846),
     ('gradient', 0.025733816),
     ('hierarchical', 0.017434655),
     ('state', 0.017009735),
     ('theory', 0.016739244),
     ('solutions', 0.014458858),
     ('selection', 0.013096148),
     ('matching', 0.012894099),
     ('swarm', 0.011814806),
     ('evolutionary', 0.010974078)]),
   (1,
    [('methods', 0.06392525),
     ('reduce', 0.039986577),
     ('based', 0.039035093),
     ('flow', 0.017953),
     ('sampling', 0.017839389),
     ('scheduler', 0.013098579),
     ('self', 0.013049646),
     ('programming', 0.01203387),
     ('hybrid', 0.011340107),
     ('ns', 0.0109894825)]),
   (2,
    [('algorithms', 0.25104824),
     ('learning', 0.051721647),
     ('machine', 0.02968633),
     ('search', 0.018754015),
     ('process', 0.011692416),
     ('graph', 0.009619729),
     ('inference', 0.009231723),
     ('fuzzy', 0.008845182),
     ('framework', 0.008426597),
     ('adaptive', 0.008190718)]),
   (3,
    [('algor

In [85]:
import os
import json
import numpy as np

# Создание директории для сохранения файлов
def create_results_directory(directory_name="results"):
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)
    return directory_name

# Кастомный JSON-сериализатор для обработки float32 и других нестандартных типов
def custom_json_serializer(obj):
    if isinstance(obj, np.float32) or isinstance(obj, np.float64):
        return float(obj)
    if isinstance(obj, np.int32) or isinstance(obj, np.int64):
        return int(obj)
    raise TypeError(f"Object of type {type(obj)} is not JSON serializable")

# Сохранение данных в формате JSON
def save_to_json(data, file_name):
    with open(file_name, "w", encoding="utf-8") as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4, default=custom_json_serializer)

# Сохранение результатов тематического моделирования
def save_results_to_json(topic_keywords, optimal_params, topic_coherences, topic_trends):
    results_dir = create_results_directory()

    # Сохранение ключевых слов для каждой категории и года
    save_to_json(topic_keywords, os.path.join(results_dir, "topic_keywords1.json"))
    print(f"Ключевые слова сохранены в {os.path.join(results_dir, 'topic_keywords1.json')}")

    # Сохранение оптимальных параметров моделей
    save_to_json(optimal_params, os.path.join(results_dir, "optimal_params1.json"))
    print(f"Оптимальные параметры сохранены в {os.path.join(results_dir, 'optimal_params1.json')}")

    # Сохранение когерентности для каждой категории и года
    save_to_json(topic_coherences, os.path.join(results_dir, "topic_coherences1.json"))
    print(f"Когерентность сохранена в {os.path.join(results_dir, 'topic_coherences1.json')}")

    # Сохранение трендов тем
    save_to_json(topic_trends, os.path.join(results_dir, "topic_trends1.json"))
    print(f"Тренды тем сохранены в {os.path.join(results_dir, 'topic_trends1.json')}")

# Вызов функции сохранения результатов
save_results_to_json(topic_keywords, optimal_params, topic_coherences, topic_trends)


Ключевые слова сохранены в results\topic_keywords1.json
Оптимальные параметры сохранены в results\optimal_params1.json
Когерентность сохранена в results\topic_coherences1.json
Тренды тем сохранены в results\topic_trends1.json
